## 128. 最长连续序列

In [2]:
from typing import List, Optional, Tuple, Set
from collections import defaultdict, Counter
from functools import lru_cache

In [5]:
class Solution:
    def longestConsecutive(self, nums: List[int]) -> int:
        nums, ans = set(nums), 0
        for n in nums:
            if n - 1 in nums: continue
            cur = n
            while cur + 1 in nums: cur += 1
            ans = max(ans, cur - n + 1)
        return ans

### 核心： 若 n - 1 也在集合中则跳过, 否则依次寻找连续的值。这样才能达到O(n)的复杂度

没有必要用并查集

## 2316. 统计无向图中无法相互到达点对数

In [57]:
class Solution:
    def countPairs(self, n: int, edges: List[List[int]]) -> int:
        def find(x):
            if fa[x] != x: fa[x] = find(fa[x])
            return fa[x]
        
        def union(x, y):
            fx, fy = find(x), find(y)
            if fx != fy:
                if rank[fx] > rank[fy]:
                    fx, fy = fy, fx
                fa[fx] = fy
                if rank[fx] == rank[fy]: rank[fy] += 1    
        
        fa = [i for i in range(n)]
        rank = [1] * n
        for a, b in edges: union(a, b)
        for i in range(n): fa[i] = find(i)
        nums, pres, cnt = [], [], Counter(fa)
        pre = 0
        for key in cnt: 
            pre += cnt[key]
            nums.append(cnt[key])
            pres.append(pre)
        ans = 0
        for i in range(len(nums)):
            ans += nums[i] * (pres[-1] - pres[i])
        
        
        
        # return nums, pres, ans
        return ans
        # return fa

In [58]:
n = 11
edges = [[5,0],[1,0],[10,7],[9,8],[7,2],[1,3],[0,2],[8,5],[4,6],[4,2]]

In [59]:
s = Solution()
s.countPairs(n, edges)

0

## 130. 被围绕的区域

换个角度, 就是看联通的"O"区域是否能够到达边界, 不能到达的都变成 "X"

In [61]:
class UF:
    def __init__(self, n, r, c):
        self.fa = [i for i in range(n)]
        self.rank = [1] * n
        self.lb = [-1] * n
        self.hb = [-1] * n
        self.r = r
        self.c = c
        
        
    def find(self, x): 
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx != fy:
            if self.rank[fx] > self.rank[fy]:
                fx, fy = fy, fx
            self.fa[fx] = fy
            if self.rank[fx] == self.rank[fy]: self.rank[fy] += 1
            row, col = divmod(fx, self.c)
            self.lb[fy] = min(self.lb[fy], min(row, col))
            self.hb[fy] = max(self.hb[fy], max(row, col))
            
            
class Solution:
    def solve(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        r, c = len(board), len(board[0])
        uf = UF(r * c, r, c)
        for i in range(r):
            for j in range(c):
                if board[i][j] == "O":
                    x = i * r + j
                    for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                        ni, nj = i + di, j + dj
                        if 0 <= ni < r and 0 <= nj < c:
                            if board[ni][nj] == "O":
                                y = ni * r + nj
                                uf.union(x, y)
        
        return uf.fa, uf.lb, uf.hb

In [62]:
board = [["X","X","X","X"],["X","O","O","X"],["X","X","O","X"],["X","O","X","X"]]

In [63]:
s = Solution()
s.solve(board)

([0, 1, 2, 3, 4, 6, 6, 7, 8, 9, 6, 11, 12, 13, 14, 15],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1])

这题用并查集有些麻烦，而且用了本质上还是要dfs或bfs。

**思路：遍历一遍矩阵，以在边界上的 “O” 为起点的 “O” 区域标记。再次遍历，若遇到没有标记的 “O” 则把他变成 “X”**

### DFS

In [69]:
class Solution:
    def solve(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        def dfs(x, y):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if board[nx][ny] == "O" and mark[nx][ny] == 0:
                        mark[nx][ny] = 1
                        dfs(nx, ny)
        
        m, n = len(board), len(board[0])
        mark = [[0] * n for _ in range(m)]
        for i in range(m):
            if board[i][0] == "O":
                mark[i][0] = 1
                dfs(i, 0)
            if board[i][n - 1] == "O":
                mark[i][n - 1] = 1
                dfs(i, n - 1)
        for i in range(n):
            if board[0][i] == "O":
                mark[0][i] = 1
                dfs(0, i)
            if board[m - 1][i] == "O":
                mark[m - 1][i] = 1
                dfs(m - 1, i)
        
        for i in range(m):
            for j in range(n):
                if board[i][j] == "O" and mark[i][j] == 0:
                    board[i][j] = "X"
        
        return board

In [70]:
board = [["X","X","X","X"],["X","O","O","X"],["X","X","O","X"],["X","O","X","X"]]

In [71]:
s = Solution()
s.solve(board)

[['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X'],
 ['X', 'O', 'X', 'X']]

### BFS

In [72]:
class Solution:
    def solve(self, board: List[List[str]]) -> None:
        if not board:
            return
        
        n, m = len(board), len(board[0])
        que = collections.deque()
        for i in range(n):
            if board[i][0] == "O":
                que.append((i, 0))
                board[i][0] = "A"
            if board[i][m - 1] == "O":
                que.append((i, m - 1))
                board[i][m - 1] = "A"
        for i in range(m - 1):
            if board[0][i] == "O":
                que.append((0, i))
                board[0][i] = "A"
            if board[n - 1][i] == "O":
                que.append((n - 1, i))
                board[n - 1][i] = "A"
        
        while que:
            x, y = que.popleft()
            for mx, my in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
                if 0 <= mx < n and 0 <= my < m and board[mx][my] == "O":
                    que.append((mx, my))
                    board[mx][my] = "A"
        
        for i in range(n):
            for j in range(m):
                if board[i][j] == "A":
                    board[i][j] = "O"
                elif board[i][j] == "O":
                    board[i][j] = "X"

- 时间复杂度: O(mn)
- 空间复杂夫: O(mn)

## 1254. 统计封闭岛屿的数目 

感觉也不用并查集

先遍历一遍矩阵把和边相连的陆地区域都变成水，这样就只剩下岛屿，再循环dfs或bfs将联通的岛屿再变成水，直到矩阵中没有陆地，那么循环的次数就是岛屿的数目。

In [79]:
class Solution:
    def closedIsland(self, grid: List[List[int]]) -> int:
        def dfs(x, y):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if grid[nx][ny] == 0:
                        grid[nx][ny] = 1
                        dfs(nx, ny)
        
        m, n = len(grid), len(grid[0])
        for i in range(m):
            if grid[i][0] == 0: 
                grid[i][0] = 1
                dfs(i, 0)
            if grid[i][n - 1] == 0: 
                grid[i][n - 1] = 1
                dfs(i, n - 1)
        for i in range(n):
            if grid[0][i] == 0: 
                grid[0][i] = 1
                dfs(0, i)
            if grid[m - 1][i] == 0: 
                grid[m - 1][i] = 1
                dfs(m - 1, i)
        
        ans = 0
        for i in range(m):
            for j in range(n):
                if grid[i][j] == 0:
                    ans += 1
                    dfs(i, j)
        return ans

In [84]:
grid = [[1,1,1,1,1,1,1],[1,0,0,0,0,0,1],[1,0,1,1,1,0,1],[1,0,1,0,1,0,1],[1,0,1,1,1,0,1],[1,0,0,0,0,0,1],[1,1,1,1,1,1,1]]

In [85]:
s = Solution()
s.closedIsland(grid)

2

- 时间复杂度：O(mn)
- 空间复杂度：O(mn)

## 695. 岛屿的最大面积

In [123]:
class Solution:
    def maxAreaOfIsland(self, grid: List[List[int]]) -> int:
        def dfs(x, y):
            cnt = 1  # 设为 1 比设为 0 要好
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if grid[nx][ny] == 1:
                        grid[nx][ny] = 0
                        cnt += dfs(nx, ny)
            return cnt
        
        m, n = len(grid), len(grid[0])
        ans = 0
        for i in range(m):
            for j in  range(n):
                if grid[i][j] == 1:
                    grid[i][j] = 0
                    ans = max(ans, dfs(i, j) + 1)
        return ans

In [121]:
grid = [[0,0,0,0,0,0,0,0]]

In [122]:
s = Solution()
s.maxAreaOfIsland(grid)

0

## 1905. 统计子岛屿

In [155]:
class Solution:
    def countSubIslands(self, grid1: List[List[int]], grid2: List[List[int]]) -> int:
        def dfs(x, y):
            ans = 1
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if grid2[nx][ny] == 1:
                        grid2[nx][ny] = 0
                        ans &= (dfs(nx, ny) and grid1[nx][ny])  # 注意这里的短路，要先执行dfs
            # res = ans & grid1[x][y] == 1
            # if res: print(x, y)
            
            return ans & grid1[x][y] == 1
        
        m, n = len(grid2), len(grid2[0])
        num = 0
        for i in range(m):
            for j in range(n):
                if grid2[i][j] == 1:
                    grid2[i][j] = 0
                    num += dfs(i, j)
        
        return num

In [156]:
grid1 = [[1,1,1,0,0],[0,1,1,1,1],[0,0,0,0,0],[1,0,0,0,0],[1,1,0,1,1]]
grid2 = [[1,1,1,0,0],[0,0,1,1,1],[0,1,0,0,0],[1,0,1,1,0],[0,1,0,1,0]]

In [157]:
s = Solution()
s.countSubIslands(grid1, grid2)

3

## 417. 太平洋大西洋水流问题

In [34]:
class Solution:
    def pacificAtlantic(self, heights: List[List[int]]) -> List[List[int]]:
        m, n = len(heights), len(heights[0])

        def search(starts: List[Tuple[int, int]]) -> Set[Tuple[int, int]]:
            visited = set()
            def dfs(x: int, y: int):
                if (x, y) in visited:
                    return
                visited.add((x, y))
                for nx, ny in ((x, y + 1), (x, y - 1), (x - 1, y), (x + 1, y)):
                    if 0 <= nx < m and 0 <= ny < n and heights[nx][ny] >= heights[x][y]:
                        dfs(nx, ny)
            for x, y in starts:
                dfs(x, y)
            return visited

        pacific = [(0, i) for i in range(n)] + [(i, 0) for i in range(1, m)]
        atlantic = [(m - 1, i) for i in range(n)] + [(i, n - 1) for i in range(m - 1)]
        return list(map(list, search(pacific) & search(atlantic)))

### set 的交和并: & |

In [19]:
{1, 2} & {1}

{1}

In [20]:
{1, 2} | {1}

{1, 2}

反向的思路就是：分别从岛的大西洋和太平洋的边界开始进行遍历，分别获得能够流向大西洋以及太平洋的格子的合集，再取交集即为结果。  
而正向做比较困难。

In [40]:
class Solution:
    def pacificAtlantic(self, heights: List[List[int]]) -> List[List[int]]:
        def search(starts):
            visit = set()
            def dfs(x, y):
                visit.add((x, y))
                for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < m and 0 <= ny < n and (nx, ny) not in visit and heights[x][y] <= heights[nx][ny]:
                        dfs(nx, ny)
            for x, y in starts: dfs(x, y)
            return visit
        
        m, n = len(heights), len(heights[0])
        pacific = [(0, i) for i in range(n)] + [(i, 0) for i in range(m)]
        atlantic = [(m - 1, i) for i in range(n)] + [(i, n - 1) for i in range(m - 1)]
        return list(map(list, search(pacific) & search(atlantic)))

In [41]:
heights = [[2,1],[1,2]]

In [42]:
s = Solution()
s.pacificAtlantic(heights)

[[0, 1], [1, 0], [0, 0], [1, 1]]

- 时间复杂度: O(mn)
- 空间复杂度: O(mn)

## 733. 图像渲染

In [45]:
class Solution:
    def floodFill(self, image: List[List[int]], sr: int, sc: int, color: int) -> List[List[int]]:
        def dfs(x, y):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if image[nx][ny] == old and image[nx][ny] != color:
                        image[nx][ny] = color
                        dfs(nx, ny)
        m, n, old = len(image), len(image[0]), image[sr][sc]
        image[sr][sc] = color
        dfs(sr, sc)
        return image

## 1020. 飞地的数量

In [2]:
from typing import List
class Solution:
    def numEnclaves(self, grid: List[List[int]]) -> int:
        def dfs(x, y):
            grid[x][y] = 0
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if grid[nx][ny] == 1:
                        dfs(nx, ny)
                        
        m, n = len(grid), len(grid[0])
        for i in range(m):
            if grid[i][0] == 1:
                dfs(i, 0)
            if grid[i][n - 1] == 1:
                dfs(i, n - 1)
        for i in range(n):
            if grid[0][i] == 1:
                dfs(0, i)
            if grid[m - 1][i] == 1:
                dfs(m - 1, i)
        
        return sum(r.count(0) for r in grid)

## 684. 冗余连接

检测图中有没有环

In [13]:
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        self.rank = [1] * n
    
    def find(self, x):
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx == fy: return 0
        if self.rank[fx] > self.rank[fy]:
            fx, fy = fy, fx
        self.fa[fx] = self.fa[fy]
        if self.rank[fx] == self.rank[fy]:
            self.rank[fy] += 1
        return 1

class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        uf = UF(len(edges))
        for x, y in edges:
            ans = uf.union(x - 1, y - 1)
            if ans == 0: return [x, y]        

In [14]:
edges = [[1,2], [2,3], [3,4], [1,4], [1,5]]

In [15]:
s = Solution()
s.findRedundantConnection(edges)

[1, 4]

## 685. 冗余连接Ⅱ

### 步骤：
1. 计算入度
2. 根据入度的不同分两种情况：
    - 存在入度为 2 的点： 若删去某条边之后该图变成了一棵树，那么这条边为要返回的结果。
    - 不存在入度为 2 的点： 必然存在有向环， 找到有向环的边就是要返回的答案。

In [30]:
from typing import List

class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
    
    def find(self, x):
        return x if self.fa[x] == x else self.find(self.fa[x])
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx == fy: return 0
        self.fa[y] = x
        return 1

    
class Solution:
    def findRedundantDirectedConnection(self, edges: List[List[int]]) -> List[int]:
        def toTreeCutEdge(edge):
            uf = UF(n + 1)
            for a, b in edges:
                if [a, b] == edge: continue
                # 去掉这条边以后依然不是树，说明答案不是这条边
                if uf.union(a, b) == 0: return False
            return True
    
        n = len(edges)
        indegree = [0] * (n + 1)
        for _, e in edges: indegree[e] += 1
        
        if 2 in indegree:
            target = indegree.index(2)
            for i in range(n - 1, -1, -1):
                # 入度存在 2， 则要删去入度为 2 的点连接的一条边
                if edges[i][1] == target and toTreeCutEdge(edges[i]):
                    return edges[i]
            
        
        # 入度不存在 2 ，图中存在有向环，找到最后出现的有向环的边即可
        uf = UF(n + 1)
        # for a, b in edges: uf.union(a, b)  # 这里注意不能一开始就全合并，不然很多边的祖先都是同一个
        for a, b in edges:
            if uf.find(a) == uf.find(b):  # 先判断，再合并
                return [a, b]
            uf.union(a, b)

In [31]:
edges = [[1,2],[2,3],[3,4],[4,1],[1,5]]

In [32]:
s = Solution()
s.findRedundantDirectedConnection(edges)

[4, 1]

## 1162. 地图分析

曼哈顿距离不就是遍历的步数吗

In [28]:
from collections import defaultdict
from typing import List
class Solution:
    def maxDistance(self, grid: List[List[int]]) -> int:
        def dfs(x, y, step):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if grid[nx][ny] == 0 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        dist[(nx, ny)] = step if (nx, ny)  not in dist else min(dist[(nx, ny)], step)
                        dfs(nx, ny, step + 1)
                        visit[nx][ny] = 1
        
        n = len(grid)
        dist = dict()
        visit = [[1] * n for _ in range(n)]
        
        for i in range(n):
            for j in range(n):
                if grid[i][j] == 1:
                    dfs(i, j, 1)
        ans = 0
        for key in dist: ans = max(ans, dist[key])
        
        return ans

In [37]:
grid = [[1,0,1],[0,0,0],[1,0,1]]

In [38]:
s = Solution()
s.maxDistance(grid)

2

In [35]:
sum(map(sum, grid))

4

这样做复杂度太高，得多源bfs

### 多源bfs做法

适用于多个等同的起点，实际上就是把所有root节点都先入栈。

In [36]:
from collections import deque
class Solution:
    def maxDistance(self, grid: List[List[int]]) -> int:
        stk = deque()
        n = len(grid)
        for i in range(n):
            for j in range(n):
                if grid[i][j] == 1: stk.append((i, j, 0))
        l = len(stk)
        if l == 0 or l == n * n: return -1
    
        visit = [[1] * n for _ in range(n)]
        while stk:
            cx, cy, step = stk.popleft()
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = cx + dx, cy + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if grid[nx][ny] == 0 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        stk.append((nx, ny, step + 1))
        return step

### 通俗解释： 陆地逐渐扩张，直到铺满整个区域！

## 42. 接雨水

### 1. dp

一个格子能够接住雨水的最大高度为两边最大高度的最小值。

In [62]:
class Solution:
    def trap(self, height: List[int]) -> int:
        def rightmax(idx):
            if idx in memo: return memo[idx]
            if idx == n - 1: return height[n - 1]
            memo[idx] = max(height[idx], rightmax(idx + 1))
            return memo[idx]
        
        n, ans, memo = len(height), 0, {}
        leftmax = height[0]
        for i in range(n):
            leftmax = max(leftmax, height[i])
            ans += min(leftmax, rightmax(i)) - height[i]
        
        return ans

In [66]:
height = [4,2,0,3,2,5]

In [67]:
s = Solution()
s.trap(height)

9

In [65]:
class Solution:
    def trap(self, height: List[int]) -> int:
        n = len(height)
        rightmax = [0] * n
        cur = 0
        for i in range(n - 1, -1, -1):
            cur = max(cur, height[i])
            rightmax[i] = cur
        leftmax = 0
        ans = 0
        for i in range(n):
            leftmax = max(leftmax, height[i])
            ans += min(leftmax, rightmax[i]) - height[i]
        return ans

### 2. 单调栈

In [68]:
class Solution:
    def trap(self, height: List[int]) -> int:
        ans = 0
        stack = list()
        n = len(height)
        
        for i, h in enumerate(height):
            while stack and h > height[stack[-1]]:
                top = stack.pop()
                if not stack:
                    break
                left = stack[-1]
                currWidth = i - left - 1
                currHeight = min(height[left], height[i]) - height[top]
                ans += currWidth * currHeight
            stack.append(i)
        
        return ans

## 542. 01矩阵

和1162类似, 多源BFS

In [10]:
from collections import deque
class Solution:
    def updateMatrix(self, mat: List[List[int]]) -> List[List[int]]:
        m, n = len(mat), len(mat[0])
        stk = deque()
        # dist = [[float("Inf")] * n for _ in range(m)]
        for i in range(m):
            for j in range(n):
                if mat[i][j] == 0: 
                    # dist[i][j] = 0
                    stk.append((i, j, 0))
        
        visit = [[1] * n for _ in range(m)]
        while stk:
            x, y, step = stk.popleft()
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if mat[nx][ny] == 1 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        mat[nx][ny] = step + 1
                        stk.append((nx, ny, step + 1))
                        
        return mat

In [11]:
mat = [[0,0,0],[0,0,0],[0,0,0]]

In [12]:
s = Solution()
s.updateMatrix(mat)

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

## 1091. 二进制矩阵中的最短路径

dp吧

In [69]:
class Solution:
    def shortestPathBinaryMatrix(self, grid: List[List[int]]) -> int:
        if grid[0][0] != 0 or grid[-1][-1] != 0: return -1
        n, stk = len(grid), deque([(0, 0, 1)])
        visit = [[1] * n for _ in range(n)]
        while stk:
            x, y, step = stk.popleft()
            # print(x, y, type(x)
            if x == y == n - 1: return step
            d = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))
            for dx, dy in d:
                # print(dx, dy, type(dx))
                nx, ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if grid[nx][ny] == 0 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        stk.append((nx, ny, step + 1))
        return -1

In [70]:
grid = [[0,0,0],[1,1,0],[1,1,0]]

In [71]:
s = Solution()
s.shortestPathBinaryMatrix(grid)

4